In [8]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../")

import src.support as sp

In [9]:
# Importing a csv file that contains all the lines said in the show
lines = pd.read_csv("../data/the_office_lines.csv")
lines.tail(3)

,id,season,episode,scene,line_text,speaker,deleted
59906,59907,9,23,114,I���m happy that this was all filmed so I can ...,Phyllis,False
59907,59908,9,23,115,I sold paper at this company for 12 years. My ...,Jim,False
59908,59909,9,23,116,I thought it was weird when you picked us to m...,Pam,False


In [10]:
# Function that prints the shape, columns, missing values, data types, head, and describe of a dataframe
sp.analyze_dataframe(lines)

Shape: (59909, 7)

Columns: Index(['id', 'season', 'episode', 'scene', 'line_text', 'speaker', 'deleted'], dtype='object')

Missing values: id           0
season       0
episode      0
scene        0
line_text    0
speaker      0
deleted      0
dtype: int64

Data types: id            int64
season        int64
episode       int64
scene         int64
line_text    object
speaker      object
deleted        bool
dtype: object

Head:    id  season  episode  scene  \
0   1       1        1      1   
1   2       1        1      1   
2   3       1        1      1   
3   4       1        1      1   
4   5       1        1      1   

                                           line_text  speaker  deleted  
0  All right Jim. Your quarterlies look very good...  Michael    False  
1         Oh, I told you. I couldn't close it. So...      Jim    False  
2  So you've come to the master for guidance? Is ...  Michael    False  
3         Actually, you called me in here, but yeah.      Jim    False  
4   

In [11]:
# Dropping deleted scenes (where values for "deleted" are True)
lines = lines[(lines["deleted"] == False)]
lines.head()

,id,season,episode,scene,line_text,speaker,deleted
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False


In [12]:
# Making all the speaker names lowercase, in order to identify possible repeated values
lines['speaker'] = lines.speaker.str.lower()

In [13]:
lines.loc[[5]]

,id,season,episode,scene,line_text,speaker,deleted
5,6,1,1,2,"[on the phone] Yes, I'd like to speak to your ...",michael,False


In [14]:
# Creating a new column called "action", and extracting the values on the line_text column that are inside square brackets
lines["action"] = lines["line_text"].str.extract("(\s*\[.*?\]\s*)", expand=True)
# Removing the text that I just extracted from the "line_text" column
lines["line_text"] = lines["line_text"].str.replace(r"(\s*\[.*?\]\s*)", " ", regex=True)

In [15]:
# Here you can see how the fifth row has changed
lines.loc[[5]]

,id,season,episode,scene,line_text,speaker,deleted,action
5,6,1,1,2,"Yes, I'd like to speak to your office manager...",michael,False,[on the phone]


In [16]:
lines.loc[[59906]]

,id,season,episode,scene,line_text,speaker,deleted,action
59906,59907,9,23,114,I���m happy that this was all filmed so I can ...,phyllis,False,NaN


In [17]:
# Replacing "���" for quote marks
lines["line_text"] = lines["line_text"].str.replace("���", "'")

In [18]:
# Here you can see how this row has changed
lines.loc[[59906]]

,id,season,episode,scene,line_text,speaker,deleted,action
59906,59907,9,23,114,I'm happy that this was all filmed so I can re...,phyllis,False,NaN


In [19]:
# There are quite some errors when we observe unique values. For example, Michael also appears as "Michel" or "Micheal"
lines["speaker"].unique()

array(['michael', 'jim', 'pam', 'dwight', 'jan', 'michel', 'todd packer',
       'phyllis', 'stanley', 'oscar', 'angela', 'kevin', 'ryan', 'man',
       'roy', 'mr. brown', 'toby', 'kelly', 'meredith', 'travel agent',
       'man on phone', 'everybody', 'lonny', 'darryl', 'teammates',
       'michael and dwight', 'warehouse worker', 'madge', 'worker',
       'katy', 'guy at bar', 'other guy at bar', 'pam and jim',
       'employee', "chili's employee", "kevin's computer",
       'warehouse guy', 'warehouse guys', 'video', 'man in video',
       'actor', 'redheaded actress', "mr. o'malley", 'albiny',
       "pam's mom", 'carol', 'bill', 'everyone', 'crowd', 'song',
       'dwight and michael', 'sherri', 'creed', 'devon', 'children',
       'kid', 'hank the security guard', 'ira', "ryan's voicemail",
       'master', 'christian', 'hostess', 'michael and christian',
       'sadiq (it guy)', 'mark', 'improv teacher', 'mary-beth',
       'girl acting pregnant', 'actress', 'michael and jim',

In [20]:
# Removing text in brackets from the speaker column
lines["speaker"] = lines["speaker"].str.replace(r"(\s*\[.*?\]\s*)", " ", regex=True)

In [21]:
# Replacing the % character with %% in the line_text column, just to make sure we don't get errors when inserting the data
# into SQL
lines["line_text"] = lines["line_text"].replace('%','%%', regex=True)

In [22]:
# Replacing any occurrences of the string "mic" preceded and/or followed by any number of alphabet letters. This will help replacing
# all the misspelled Michaels
lines["speaker"] = lines["speaker"].str.replace('[a-zA-Z]*mic[a-zA-Z]*', 'michael', regex=True)

In [23]:
# Strip white spaces from each value in the object columns
df_obj = lines.select_dtypes(['object'])
lines[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [24]:
lines["speaker"] = lines["speaker"].str.capitalize()
lines.head()

,id,season,episode,scene,line_text,speaker,deleted,action
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False,NaN
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False,NaN
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False,NaN
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False,NaN
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False,NaN


In [25]:
lines = lines.rename(columns = {"id": "line_id", "speaker": "first_name"})
lines.head()

,line_id,season,episode,scene,line_text,first_name,deleted,action
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False,NaN
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False,NaN
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False,NaN
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False,NaN
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False,NaN


### From here, code purpose is to create an episode_id column in order to match other episode_id columns in the others csv

In [26]:
lines1 = lines.groupby(['season', 'episode']).size().reset_index(name='freq')
lines1.head()

,season,episode,freq
0,1,1,229
1,1,2,217
2,1,3,244
3,1,4,265
4,1,5,235


In [27]:
lines1.index = lines1.index.set_names(["episode_id"])
lines1 = lines1.reset_index()
lines1.head()

,episode_id,season,episode,freq
0,0,1,1,229
1,1,1,2,217
2,2,1,3,244
3,3,1,4,265
4,4,1,5,235


In [28]:
# As you can see, the goal was to create this episode_id column, having an individual id for every episode (like I have on every csv
# in this project)
lines1.tail()

,episode_id,season,episode,freq
181,181,9,19,273
182,182,9,20,234
183,183,9,21,382
184,184,9,22,501
185,185,9,23,522


In [29]:
# Merging the two dataframes
left_merged = pd.merge(lines,lines1, how="left", on=["season", "episode"])

In [30]:
left_merged.head()

,line_id,season,episode,scene,line_text,first_name,deleted,action,episode_id,freq
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False,NaN,0,229
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False,NaN,0,229
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False,NaN,0,229
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False,NaN,0,229
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False,NaN,0,229


In [31]:
# Dropping the "fred" column, as it is irrelevant for the analysis
left_merged = left_merged.drop("freq", axis=1)

In [32]:
left_merged.tail()

,line_id,season,episode,scene,line_text,first_name,deleted,action,episode_id
57968,59905,9,23,112,It all seems so very arbitrary. I applied for ...,Creed,False,[chuckles],185
57969,59906,9,23,113,I just feel lucky that I got a chance to share...,Meredith,False,NaN,185
57970,59907,9,23,114,I'm happy that this was all filmed so I can re...,Phyllis,False,NaN,185
57971,59908,9,23,115,I sold paper at this company for 12 years. My ...,Jim,False,NaN,185
57972,59909,9,23,116,I thought it was weird when you picked us to m...,Pam,False,NaN,185


In [33]:
left_merged.to_csv('../data/lines.csv', index=False)